In [1]:
import os
import sys
import pandas as pd
import numpy as np
import glob
import time
import gget
import scipy
import matplotlib.patches as patches
from scipy.sparse import csr_matrix
import anndata as an
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import fisher_exact
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from pycirclize import Circos
from scipy.interpolate import splprep, splev
import networkx as nx
import random
from importlib import reload
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import warnings
from itertools import combinations
import ot
from scipy.spatial.distance import pdist, squareform
from matplotlib.colors import ListedColormap

import surprise as sup

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

"""WARNING: no warnings"""
warnings.filterwarnings("ignore")

# local imports
import anndata_utils as anntools

source_path = os.path.abspath("../source/")
sys.path.append(source_path)
import centrality as central
import matrix
import utils as ut
import plotting as plt2

# Load pathways

In [2]:
def load_pathway(fpath):
    """
    Loads an Enrichr-like database file into a boolean DataFrame.

    Args:
        fpath (str): Path to the Enrichr-like database file.

    Returns:
        pandas.DataFrame: A boolean DataFrame where:
            - Index: Genes
            - Columns: Pathways
            - Values: True if the gene is in the pathway, False otherwise.
    """

    result = []
    with open(fpath,  encoding='utf-8') as f:
        for line in f:
            split_line = [x for x in line.strip().split('\t') if x]  # Remove empty strings directly

            row = {'label': split_line[0]}
            for gene in split_line[1:]:
                row[gene] = 1

            result.append(row)

    df = pd.DataFrame(result)
    df = df.fillna(0.0).set_index('label').astype(bool).T  # Chained operations for clarity

    return df

fpath = "../../ONT-single-cell/resources/PanglaoDB_Augmented_2021.txt"
pdf = load_pathway(fpath)
stem_genes = list(pdf[pdf['Embryonic Stem Cells']].index)
stem_genes = [x.title() for x in stem_genes]
stem_genes[:10]

['Gjb1',
 'Amotl2',
 'Yap1',
 'Fbln1',
 'Uaca',
 'Antxr1',
 'Fermt2',
 'Serpinh1',
 'Pls3',
 'Gpx8']

In [3]:
""" LOAD the TF list """
fpath = "/nfs/turbo/umms-indikar/shared/projects/twin_cell/data/b_matrix/SCENIC/scenic/tf_lists/allTFs_mm.txt"
tf_list = [x.strip() for x in open(fpath)]

# Load expression

In [4]:
fpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/expression_table/rna_table.parquet"
tdf = pd.read_parquet(fpath)
print(f"(raw) {tdf.shape=}")
print(f"(filtered) {tdf.shape=}")

expression_map = dict(zip(tdf['gene_name'].values, tdf['TPM'].values))

tdf.sample(5)

(raw) tdf.shape=(51883, 19)
(filtered) tdf.shape=(51883, 19)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation,ens_gene_id,gene_name
6351,ENSMUSG00000027878.11,"ENSMUST00000079812.7,ENSMUST00000198324.1",10500.0,10248.24,1047.0,7.57,5.64,1047.0,0.0,7.26,5.68,6.803730e+00,7.740130,0.021901,5.310430e+00,6.042870,0.021852,ENSMUSG00000027878,Notch2
51879,ENSMUSG00000118390.1,ENSMUST00000237599.1,2368.0,2116.24,0.0,0.00,0.00,0.0,0.0,0.03,0.03,2.025230e-07,0.099766,0.653571,1.580150e-07,0.078040,0.653555,ENSMUSG00000118390,Gm50102
39641,ENSMUSG00000105074.1,ENSMUST00000200605.1,466.0,214.41,0.0,0.00,0.00,0.0,0.0,0.33,0.26,1.976270e-06,0.988187,0.656864,1.542960e-06,0.772439,0.656955,ENSMUSG00000105074,Gm43818
18075,ENSMUSG00000064737.1,ENSMUST00000082803.1,111.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,ENSMUSG00000064737,Gm23056
39366,ENSMUSG00000104759.2,ENSMUST00000197110.2,386.0,135.18,0.0,0.00,0.00,0.0,0.0,0.52,0.41,6.959990e-06,1.541170,0.651119,5.437820e-06,1.205300,0.651176,ENSMUSG00000104759,Ranbp2-ps12


# Load core scores

In [5]:
resolution = 1000000
score_column = 'global_hge_logexp_RNA_weighted'
core_threshold_quantile = 0.75

fpath = f"/scratch/indikar_root/indikar1/shared_data/higher_order/global_core_score/population_mESC_{resolution}_scores.csv"
scores = pd.read_csv(fpath)
print(f"{scores.shape=}")

threshold = np.quantile(scores[score_column].fillna(0.0), core_threshold_quantile)
scores['is_core'] = scores[score_column] > threshold

scores.head()

scores.shape=(2431, 36)


,bin_name,bin_index,bin_start,bin_end,bin,chrom,chrom_bin,degree,genes,n_genes,...,ce_pagerank,hge_singular_vector_1,hge_logexp_unweighted,hge_logexp_degree_weighted,hge_logexp_RNA_weighted,hge_logexp_ATAC_weighted,global_singular_vector_1,global_hge_logexp_unweighted,global_hge_logexp_RNA_weighted,is_core
0,chr1:3,2235,3000000,4000000,3,1,3,2953,Gm37363;Gm37381;Gm7341;Gm38148;Gm37180;Gm19938...,15,...,0.117995,0.000000,0.148099,0.000000,0.174370,0.222380,0.330855,0.501567,0.167764,False
1,chr1:4,964,4000000,5000000,4,1,4,3331,Gm7369;A930006A01Rik;Gm37381;Gm38076;Mrpl15;Gm...,22,...,0.372869,0.103845,0.239187,0.281148,0.468270,0.560302,0.412235,0.609261,0.369136,True
2,chr1:5,1823,5000000,6000000,5,1,5,3341,Atp6v1h;Gm17101;Gm16041;Rgs20;Gm36965;Gm38264;...,12,...,0.224249,0.057865,0.647027,0.201096,0.243125,0.405148,0.425601,0.528269,0.216383,False
3,chr1:6,1288,6000000,7000000,6,1,6,3422,St18;Gm7449;Gm19214;Gm37108;Gm2147;Gm19026;473...,10,...,0.534940,0.172229,0.760979,0.401754,0.250762,0.521777,0.448264,0.492633,0.258762,False
4,chr1:7,144,7000000,8000000,7,1,7,3289,Gm23274;Gm5694;Rps2-ps2;Gm37225;Gm37489;Gm2690...,18,...,0.099600,0.053319,0.000000,0.160235,0.190846,0.204818,0.414267,0.510526,0.204855,False


In [6]:
fpath = f"/scratch/indikar_root/indikar1/shared_data/higher_order/duplicate_table/singlecell_mESC_{resolution}_dupes.parquet"
dupes = pd.read_parquet(fpath)
print(f"{dupes.shape=}")

cells = dupes[dupes['unique']]
print(f"(raw) {cells.shape=}")
cells = cells[['read_id', 'cell_id']].drop_duplicates()
print(f"(filtered) {cells.shape=}")
cell_map = dict(zip(cells['read_id'].values, cells['cell_id'].values))

dupes.head()

dupes.shape=(10902736, 8)
(raw) cells.shape=(339359, 8)
(filtered) cells.shape=(339359, 2)


KeyError: 'unique'

In [8]:
dupes.columns

Index(['read_id', 'mapping_quality', 'cell_id', 'exactly_unique',
       'approximately_unique', 'read_group', 'unique', 'chrom'],
      dtype='object')

# single-cell transcription factories

In [ ]:
""" LOAD the genes """
fpath = f"/scratch/indikar_root/indikar1/shared_data/higher_order/lightweight/singlecell_mESC_{resolution}_gdf.parquet"
df = pd.read_parquet(fpath)
df['is_tf'] = df['gene_name'].isin(tf_list)
df['is_marker'] = df['gene_name'].isin(stem_genes)


""" MERGE the core scores """
merge_columns = [
    'bin_name',
    'ATACSeq_1',
    'CTCF',
    'H3K27ac', 
    'H3K27me3',
    'RNA_5',
    'global_hge_logexp_RNA_weighted'
]

df = pd.merge(
    df, scores[merge_columns].fillna(0.0), how="left",
)

""" MERGE expression information and cell_ids """
df['expression'] = df['gene_name'].map(expression_map)
df['cell_id'] = df['read_name'].map(cell_map)
df['unique'] = df['read_name'].map(dup_map)


print(f"{df.shape=}")
df.head()

# define single-cell transcription clusters

In [ ]:
order_threshold = 3
expression_threshold = 0.5

In [ ]:
break

In [ ]:
# get hyperedges
def get_hyperedge(genes):
    return "-".join(sorted(genes))

gx = df.groupby(['read_name', 'basename']).agg(
    genes=('gene_name', get_hyperedge)
).reset_index(drop=False)

print(f"{gx.shape=}")